In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/27 11:44:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
FILE = '../../de_bootcamp/week5/data/raw/fhvhv/2021/02/fhvhv_tripdata_2021-02.csv'
TARGET_DIR= '../../de_bootcamp/week5/data/parquet/'

In [6]:
from pyspark.sql import types

In [45]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [48]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(FILE)

In [49]:
df = df.repartition(24)

In [50]:
df.write.parquet(f'{TARGET_DIR}test', mode='overwrite')

In [7]:
df = spark.read.parquet(f'{TARGET_DIR}test')

In [8]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [9]:
df.groupBy('dispatching_base_num').count().orderBy('count', ascending = False).show()  

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [10]:
z_file = '../../de_bootcamp/week5/data/taxi+_zone_lookup.csv'
target_zones =  '../../de_bootcamp/week5/data/parquet/zones'

In [25]:
zone_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [28]:
df_zones = spark.read.option("header", "true").schema(zone_schema).csv(z_file)

In [29]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [30]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [31]:
df_zones= df_zones.repartition(24)

In [11]:
df_zones = df_zones.write.parquet(target_zones)

NameError: name 'df_zones' is not defined

In [12]:
df_z = spark.read.parquet(target_zones)

In [13]:
df_z.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|       127|    Manhattan|              Inwood|   Boro Zone|
|       206|Staten Island|Saint George/New ...|   Boro Zone|
|        62|     Brooklyn| Crown Heights South|   Boro Zone|
|       186|    Manhattan|Penn Station/Madi...| Yellow Zone|
|       244|    Manhattan|Washington Height...|   Boro Zone|
|       110|Staten Island|    Great Kills Park|   Boro Zone|
|       122|       Queens|              Hollis|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|        16|       Queens|             Bayside|   Boro Zone|
|       238|    Manhattan|Upper West Side N...| Yellow Zone|
|        77|     Brooklyn|East New York/Pen...|   Boro Zone|
|       131|       Queens|     Jamaica Estates|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|       190|     Brookly

In [14]:
df_z.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [15]:
from pyspark.sql.functions import concat,col, concat_ws

In [63]:
df2=df.select(concat_ws('/',df.PULocationID,df.DOLocationID).alias("pair_loc"))

In [64]:
df2.show()

+--------+
|pair_loc|
+--------+
| 163/235|
| 225/181|
|  260/95|
|  235/60|
|   55/55|
|   74/81|
| 239/231|
|  95/196|
| 241/265|
|  90/137|
|     7/7|
| 231/246|
|  108/29|
|  11/112|
| 229/234|
|  10/218|
|  49/232|
|  125/17|
|   47/68|
|  41/223|
+--------+
only showing top 20 rows



In [67]:
df.sort('PULocationID', 'DOLocationID').show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02875|2021-02-14 06:46:37|2021-02-14 06:49:47|           1|           1|   null|
|           HV0004|              B02800|2021-02-09 19:37:11|2021-02-09 20:10:00|           1|           4|   null|
|           HV0004|              B02800|2021-02-27 17:45:56|2021-02-27 18:31:31|           1|           4|   null|
|           HV0004|              B02800|2021-02-20 15:23:25|2021-02-20 16:00:14|           1|           4|   null|
|           HV0004|              B02800|2021-02-05 08:41:49|2021-02-05 09:17:19|           1|          14|   null|
|           HV0004|              B02800|2021-02-28 21:20:25|2021-02-28 21:45:57|

In [55]:
df.groupBy('PULocationID', 'DOLocationID').count().orderBy('count', ascending = False).show()   

+------------+------------+-----+
|PULocationID|DOLocationID|count|
+------------+------------+-----+
|          76|          76|45041|
|          26|          26|37329|
|          39|          39|28026|
|          61|          61|25976|
|          14|          14|17934|
|         129|         129|14688|
|           7|           7|14688|
|          42|          42|14481|
|          37|          37|14424|
|          89|          89|13976|
|         216|         216|13716|
|          35|          35|12829|
|         132|         265|12542|
|         188|          61|11814|
|          95|          95|11548|
|          36|          37|11491|
|          37|          36|11487|
|          61|         188|11462|
|          61|         225|11342|
|         188|         188|11308|
+------------+------------+-----+
only showing top 20 rows



In [57]:
df.registerTempTable('notjoin')

In [59]:
spark.sql("""
SELECT 
concat(PULocationID, '/',DOLocationID) AS conc,
count(*) AS cnt
FROM notjoin
GROUP BY  conc
ORDER BY cnt DESC
""").show()

+-------+-----+
|   conc|  cnt|
+-------+-----+
|  76/76|45041|
|  26/26|37329|
|  39/39|28026|
|  61/61|25976|
|  14/14|17934|
|    7/7|14688|
|129/129|14688|
|  42/42|14481|
|  37/37|14424|
|  89/89|13976|
|216/216|13716|
|  35/35|12829|
|132/265|12542|
| 188/61|11814|
|  95/95|11548|
|  36/37|11491|
|  37/36|11487|
| 61/188|11462|
| 61/225|11342|
|188/188|11308|
+-------+-----+
only showing top 20 rows



In [60]:
df_3 = spark.sql("""
SELECT 
t.PULocationID, t.DOLocationID, p_z.Zone AS puZone, d_z.Zone AS doZone
FROM notjoin AS t
LEFT JOIN zones AS p_z ON p_z.LocationID = t.PULocationID
LEFT JOIN zones AS d_z ON p_z.LocationID = t.DOLocationID
""")

In [61]:
df_3.registerTempTable('result_df_3')

In [62]:
spark.sql("""
SELECT 
puZone,  doZone,
count(*) AS cnt
FROM result 
GROUP BY  puZone,  doZone
ORDER BY cnt DESC
""").show()

+-------------------+--------------------+------+
|             puZone|              doZone|   cnt|
+-------------------+--------------------+------+
|      East New York|Governor's Island...|135123|
|       Borough Park|Governor's Island...|111987|
|      East New York|              Corona| 90082|
|           Canarsie|Governor's Island...| 84078|
|Crown Heights North|Governor's Island...| 77928|
|       Borough Park|              Corona| 74658|
|           Canarsie|              Corona| 56052|
|          Bay Ridge|Governor's Island...| 53802|
|Crown Heights North|              Corona| 51952|
|      East New York|            Elmhurst| 45041|
|      East New York|     Pelham Bay Park| 45041|
|      East New York|            Gramercy| 45041|
|      East New York|       East New York| 45041|
|      East New York|Forest Park/Highl...| 45041|
|      East New York|          Mount Hope| 45041|
|      East New York|Springfield Garde...| 45041|
|      East New York|        East Chelsea| 45041|


In [22]:
df_result = df.join(df_z, df.PULocationID == df_z.LocationID)

In [23]:
df_result.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|                Zone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|       163|Manhattan|       Midtown North| Yellow Zone|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|       225| Brooklyn|  Stuyvesant Heights|   Boro Zone|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|       260|   Queens|            Woo

In [50]:
df_result.registerTempTable('nojoin')

In [56]:
spark.sql("""
SELECT 
concat(PULocationID, '/',DOLocationID) AS conc,
count(*) AS cnt
FROM result 
GROUP BY  conc
ORDER BY cnt DESC
""").show()

+-------+--------+
|   conc|     cnt|
+-------+--------+
|  76/76|11935865|
|  26/26| 9892185|
|  39/39| 7426890|
|  61/61| 6883640|
|  14/14| 4752510|
|    7/7| 3892320|
|129/129| 3892320|
|  42/42| 3837465|
|  37/37| 3822360|
|  89/89| 3703640|
|216/216| 3634740|
|  35/35| 3399685|
|  95/95| 3060220|
|188/188| 2996620|
|  17/17| 2920565|
|225/225| 2828875|
|  82/82| 2810060|
|213/213| 2787535|
|244/244| 2693725|
|181/181| 2589315|
+-------+--------+
only showing top 20 rows



In [52]:
df_res_nojoin = spark.sql("""
SELECT 
PULocationID, DOLocationID,
count(*) AS cnt
FROM result 
GROUP BY  PULocationID, DOLocationID
ORDER BY cnt DESC
""")

In [53]:
df_res_nojoin.show()

+------------+------------+--------+
|PULocationID|DOLocationID|     cnt|
+------------+------------+--------+
|          76|          76|11935865|
|          26|          26| 9892185|
|          39|          39| 7426890|
|          61|          61| 6883640|
|          14|          14| 4752510|
|           7|           7| 3892320|
|         129|         129| 3892320|
|          42|          42| 3837465|
|          37|          37| 3822360|
|          89|          89| 3703640|
|         216|         216| 3634740|
|          35|          35| 3399685|
|          95|          95| 3060220|
|         188|         188| 2996620|
|          17|          17| 2920565|
|         225|         225| 2828875|
|          82|          82| 2810060|
|         213|         213| 2787535|
|         244|         244| 2693725|
|         181|         181| 2589315|
+------------+------------+--------+
only showing top 20 rows



In [24]:
df_result = df_result.join(df_z, df_result.DOLocationID == df_z.LocationID)

In [25]:
df.registerTempTable('joined')

In [19]:
df_z.registerTempTable('zones')

In [54]:
spark.sql("""
SELECT 
*
FROM zones
WHERE LocationID=76
""").show()

+----------+--------+-------------+------------+
|LocationID| Borough|         Zone|service_zone|
+----------+--------+-------------+------------+
|        76|Brooklyn|East New York|   Boro Zone|
+----------+--------+-------------+------------+



In [26]:
df_result=spark.sql("""
SELECT 
*
FROM zones

""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|       127|    Manhattan|              Inwood|   Boro Zone|
|       206|Staten Island|Saint George/New ...|   Boro Zone|
|        62|     Brooklyn| Crown Heights South|   Boro Zone|
|       186|    Manhattan|Penn Station/Madi...| Yellow Zone|
|       244|    Manhattan|Washington Height...|   Boro Zone|
|       110|Staten Island|    Great Kills Park|   Boro Zone|
|       122|       Queens|              Hollis|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|        16|       Queens|             Bayside|   Boro Zone|
|       238|    Manhattan|Upper West Side N...| Yellow Zone|
|        77|     Brooklyn|East New York/Pen...|   Boro Zone|
|       131|       Queens|     Jamaica Estates|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|       190|     Brookly

In [27]:
# df_result = spark.sql("""
# SELECT 
# t.PULocationID, t.DOLocationID,  p_z.Borough AS puBorough, p_z.Zone AS puZone, d_z.Borough AS doBorough , d_z.Zone AS doZone
# FROM joined AS t
# INNER JOIN zones AS p_z ON p_z.LocationID = t.PULocationID
#  JOIN zones AS d_z ON p_z.LocationID = t.DOLocationID
# """)

In [31]:
df_result = spark.sql("""
SELECT 
t.PULocationID, t.DOLocationID, p_z.Zone AS puZone, d_z.Zone AS doZone
FROM joined AS t
INNER JOIN zones AS p_z ON p_z.LocationID = t.PULocationID
JOIN zones AS d_z ON p_z.LocationID = t.DOLocationID
""")

In [32]:
df_result.registerTempTable('result')

In [33]:
df_res2 = spark.sql("""
SELECT 
puZone, doZone,
count(*) AS cnt
FROM result 
GROUP BY  puZone, doZone
ORDER BY cnt DESC
""")

In [34]:
df_res2.show(truncate=False)

+-------------------+---------------------------------------------+------+
|puZone             |doZone                                       |cnt   |
+-------------------+---------------------------------------------+------+
|East New York      |Governor's Island/Ellis Island/Liberty Island|135123|
|Borough Park       |Governor's Island/Ellis Island/Liberty Island|111987|
|East New York      |Corona                                       |90082 |
|Canarsie           |Governor's Island/Ellis Island/Liberty Island|84078 |
|Crown Heights North|Governor's Island/Ellis Island/Liberty Island|77928 |
|Borough Park       |Corona                                       |74658 |
|Canarsie           |Corona                                       |56052 |
|Bay Ridge          |Governor's Island/Ellis Island/Liberty Island|53802 |
|Crown Heights North|Corona                                       |51952 |
|East New York      |Elmhurst                                     |45041 |
|East New York      |Spri

In [42]:
df_1step = df.join(df_z, df.PULocationID == df_z.LocationID,"left").withColumnRenamed("Zone","PUZone")

In [43]:
df_1step.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|              PUZone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|       163|Manhattan|       Midtown North| Yellow Zone|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|       225| Brooklyn|  Stuyvesant Heights|   Boro Zone|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|       260|   Queens|            Woo

In [45]:
df_2step = df_1step.join(df_z, df.DOLocationID == df_z.LocationID,"left").withColumnRenamed("Zone","DOZone")

In [46]:
df_2step.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+-------------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID|  Borough|              PUZone|service_zone|LocationID|      Borough|              DOZone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+---------+--------------------+------------+----------+-------------+--------------------+------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|       163|Manhattan|       Midtown North| Yellow Zone|      null|         null|                null|        null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|    

In [47]:
df_2step.registerTempTable('df_2')

In [48]:
df_res_2step = spark.sql("""
SELECT 
PUZone, DOZone,
count(*) AS cnt
FROM df_2
GROUP BY  PUZone, DOZone
ORDER BY cnt DESC
""")

In [49]:
df_res_2step.show()

+--------------------+--------------------+------+
|              PUZone|              DOZone|   cnt|
+--------------------+--------------------+------+
| Crown Heights North|                null|177801|
|       East New York|Governor's Island...|135123|
|        East Village|                null|133580|
|      Bushwick South|                null|126212|
|Central Harlem North|                null|122765|
|       East New York|                null|121918|
|             Bedford|                null|114373|
|        Borough Park|Governor's Island...|111987|
|  Stuyvesant Heights|                null|109351|
|Prospect-Lefferts...|                null|106426|
|Washington Height...|                null|105425|
|TriBeCa/Civic Center|                null|104517|
|Flatbush/Ditmas Park|                null|103116|
|             Astoria|                null| 96099|
|            Union Sq|                null| 94946|
|   East Harlem North|                null| 94867|
|          Park Slope|         